In [220]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [216]:
item = pd.read_csv("data/items.csv")
item.head()

,product_id,category,subcategory,price
0,0001c1dd-462e-453c-bac8-2fcdc0b0bc8d,A,0,264.61
1,0002278b-67e3-406e-bfc2-63c48c195e2b,A,0,248.31
2,0002c59e-df87-47a8-a27d-26b34340bee5,B,1,379.53
3,000362d7-cc2a-4912-b5ab-0b931e77942a,C,2,0.00
4,000408f0-f0d0-447f-8700-30bb7dd390b1,A,0,0.00


In [184]:
user = pd.read_csv("data/users.csv")

In [217]:
le_customer = preprocessing.LabelEncoder()
le_product = preprocessing.LabelEncoder()
user['customer_id'] = le_customer.fit_transform(user['customer_id'])
user['product_id'] = le_customer.fit_transform(user['product_id'])

In [218]:
user.head()

,customer_id,product_id,timestamp,value
0,113346,0,2021-03-22 04:45:17.308966+00:00,5
1,1121730,1,2021-03-22 04:59:26.404186+00:00,5
2,695567,2,2021-03-22 05:13:58.989935+00:00,5
3,695567,3,2021-03-22 05:13:58.991935+00:00,5
4,868593,4,2021-03-22 05:32:16.083698+00:00,5


In [221]:
train, test = train_test_split(user, test_size = 0.25, random_state = 42)

In [296]:
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam, RMSprop

In [320]:
def make_model(embedding_size, num_values, num_users, num_rows):
    product_input = layers.Input(shape=(1,), name='value')

    product_emb = layers.Embedding(output_dim = embedding_size , input_dim = len(user.customer_id.unique()) + 1 , name='product_emb')(product_input)
    product_vec = layers.Flatten(name='FlattenProduct')(product_emb)

    product_model = Model(inputs= product_input, outputs = product_vec)
    
    user_input = layers.Input(shape=(1,), name='customer_id')

    user_emb = layers.Embedding(output_dim = embedding_size, input_dim = len(user.product_id.unique()) + 1, name='user_emb')(user_input)
    user_vec = layers.Flatten(name='FlattenUser')(user_emb)

    user_model = Model(inputs = user_input, outputs = user_vec)
    
    merged = layers.Dot(name = 'dot_product', normalize = True, axes = 2)([product_emb, user_emb])
    merged_dropout = layers.Dropout(0.2)(merged)
    
    
    #dense_1 = layers.Dense(70, name = 'FullyConnected-1')(merged)
    #dropout_1 = layers.Dropout(0.2, name = 'Dropout_1')(dense_1)

    #dense_2 = layers.Dense(50, name='FullyConnected-2')(dropout_1)
    #dropout_2 = layers.Dropout(0.2, name='Dropout_2')(dense_2)

    dense_3 = layers.Dense(20,name='FullyConnected-3')(merged)
    dropout_3 = layers.Dropout(0.2,name='Dropout_3')(dense_3)

    dense_4 = layers.Dense(10,name='FullyConnected-4', activation='relu')(dropout_3)

    result = layers.Dense(1, name='result', activation="relu") (dense_4)

    adam = Adam(lr=0.001)
    model = Model([product_input, user_input], result)
    model.compile(optimizer = adam, loss= 'mean_absolute_error', metrics = ["accuracy"])
    return model, product_model, user_model

In [321]:
model, product_model, user_model = make_model(10, num_value, num_users, df_train.shape[0])

In [322]:
train.head()

,customer_id,product_id,timestamp,value
6830297,636000,863,2021-04-22 16:56:40.417467+00:00,1
12599642,709278,11605,2021-05-15 03:30:50.427634+00:00,1
8569471,2074131,212,2021-04-29 01:48:33.788175+00:00,1
9272220,1294099,28775,2021-05-01 22:17:27.536002+00:00,1
7907596,1568550,17553,2021-04-26 19:26:12.854142+00:00,1


In [319]:
train_list = list(train.value)
if 5 in train_list:
    print("yrs")

yrs


In [323]:
epochs = 1
r = model.fit(
  x = [train.customer_id.values, train.product_id.values],
  y = train.value.values,
  epochs=epochs,
  batch_size=8000,
    validation_data=(
    [test.customer_id.values, test.product_id.values],
    test.value.values
  )
)

Epoch 1/2
1388/1388 [==============================] - 263s 189ms/step - loss: 0.1261 - accuracy: 0.9392 - val_loss: 0.0515 - val_accuracy: 0.9872
Epoch 2/2
1388/1388 [==============================] - 263s 189ms/step - loss: 0.0523 - accuracy: 0.9873 - val_loss: 0.0545 - val_accuracy: 0.9872


In [312]:
#test.drop(columns = ["timestamp"], inplace = True)

In [311]:
test.tail()

,customer_id,product_id,value
7419079,1230502,96110,1
13132036,1594492,750,1
4975236,1857601,1250,1
1875128,922514,7729,1
9160480,1414191,35410,1


In [310]:
value_list = list(test["value"])
if 5 in value_list:
    print("yes")

yes


In [271]:
test.iloc[0]

customer_id    976283
product_id     115793
value               1
Name: 12557689, dtype: int64

In [ ]:
arr = model.predict([ test.customer_id.values , test.product_id.values ])

In [318]:
arr = np.round(arr)
if 5. in arr:
    print("True")

In [287]:
test.shape

(3700775, 3)

In [288]:
import collections
MOVIE_EMBEDDING_LIST = []
MOVIE_EMBED_MAP = collections.defaultdict()
UNIQUE_MOVIE_IDS = user["customer_id"].unique()
type(UNIQUE_MOVIE_IDS)

numpy.ndarray

In [289]:
UNIQUE_MOVIE_IDS[:10]

array([ 113346, 1121730,  695567,  868593,  716965, 2004067, 1719348,
        498138,  362569,   72725])

In [293]:
for _id in UNIQUE_MOVIE_IDS[:10]:
    emb = user_model.predict(np.array([_id]))
    val = list(emb.reshape(1,-1))[0]
    MOVIE_EMBEDDING_LIST.insert(_id, val)
    MOVIE_EMBED_MAP[_id] = val

MOVIE_EMBEDDING_LIST

[array([-0.01598167,  0.0084476 ,  0.00865109,  0.0132635 , -0.04414903],
       dtype=float32),
 array([-0.00467436,  0.04511095,  0.04468863,  0.01084341, -0.01994261],
       dtype=float32),
 array([0.04497014, 0.0378111 , 0.00078298, 0.00027316, 0.01748667],
       dtype=float32),
 array([ 0.02604197, -0.02542784, -0.04157677,  0.01745602,  0.00638946],
       dtype=float32),
 array([-0.02974293, -0.0357784 ,  0.0238178 , -0.0493248 ,  0.00313913],
       dtype=float32),
 array([ 0.04011121,  0.01622117, -0.03266661,  0.0081953 ,  0.03808197],
       dtype=float32),
 array([ 0.01499167, -0.04739866, -0.04962599, -0.04822024,  0.02555228],
       dtype=float32),
 array([ 0.00715816, -0.04995593,  0.02017344,  0.03460372,  0.01181942],
       dtype=float32),
 array([ 0.02132684, -0.04243025, -0.03705305,  0.04872218,  0.00115388],
       dtype=float32),
 array([-0.0145869 ,  0.03844975, -0.00018432,  0.01352631,  0.0151811 ],
       dtype=float32),
 array([-0.01598167,  0.0084476 ,  

In [292]:
TEST_USER_ID = 200
#get the embedding of this user
user_embedding = user_model.predict([TEST_USER_ID]).reshape(1,-1)[0]
knn_train_label = MOVIE_EMBEDDING_LIST

#create the KNN model
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=11)
clf.fit(MOVIE_EMBEDDING_LIST, knn_train_label)

def recommend_movies(embedding):
    distances, indices = clf.kneighbors(embedding.reshape(1, -1),  n_neighbors=10)
    indices = indices.reshape(10,1)
    df_indices = pd.DataFrame(indices, columns = ['movie_id'])
    return df_indices.merge(movies,on='movie_id',how='inner',suffixes=['_u', '_m'])['title']
  
# get the recommendation
recommend_movies(user_embedding)

ValueError: Unknown label type: 'continuous-multioutput'